In [1]:
import gzip
import pickle
from os.path import join, expanduser
import pandas as pd
import json
import numpy as np

from article_analysis.parse import get_chunk, get_articles, find_doi_chunk_map, load_ngram_dist
import article_analysis.parse_ent as aape
%load_ext autoreload
%autoreload 1
%aimport article_analysis.parse_ent

In [2]:
head = 50
verbose = True

nlp = aape.init_nlp()

input_path = expanduser('~/data/jstor/latest/')
output_path = expanduser('~/data/jstor/latest/')
prefix = 'ngrams_dict'

with open(join(output_path, 'registry_json.txt')) as file:
    registry_dict = json.loads(file.read())


fname = expanduser('~/data/jstor/latest/corpus_clean_dict.pgz')
with gzip.open(fname) as fp:
    articles_ds = pickle.load(fp)

In [10]:
head = 2
# keywords = ['hypothesis', 'hypotheses', 'table']
# keywords = ['hypothesis', 'Hypothesis', 'hypotheses', 'Hypotheses', 'proposition']
# keywords = ['table']
keywords = ['proposition', 'hypothesis']

In [4]:
all_dois_flat = [v for sublist in registry_dict.values() for v in sublist]
head = 10
head = 2
nst = []

if head > 0:
    all_dois_flat = all_dois_flat[:head]

# check boundaries
batch_size = 100
batch_size = 10
dois_batched = [all_dois_flat[i:i+batch_size] for i in range(0, len(all_dois_flat), batch_size)]

df_agg = []
flags = []
for dois_batch, j in zip(dois_batched, range(len(dois_batched))):
    if verbose:
        print('batch number {0}'.format(j))
    dch_dict = get_articles(dois_batch, registry_dict, input_path, prefix)
    for doi in dois_batch:
        ngram_order = 1
        article_ngrams = dch_dict[doi]
        ngram_positions_list = article_ngrams[ngram_order]
        
        ixs = []
        for keyword in keywords:
            if keyword in ngram_positions_list.keys():
                ixs += ngram_positions_list[keyword]
#         print('doi {0}, len ixs {1}'.format(doi, len(ixs)))
        
        carticle = articles_ds[doi]
        phrases = [phrase for phrase in carticle if any([True for keyword in keywords if keyword in phrase])]
#         print('doi {0}, len ixs {1}'.format(doi, len(phrases)))
#         phrases_start = [phrase for phrase in carticle if any([True for keyword in keywords if keyword == phrase[0]])]
        phrases_start = [phrase for phrase in carticle if any([True for keyword in keywords if keyword == phrase[0]])]

#         print('doi {0}, len ixs {1}; starts {2}.'.format(doi, len(phrases), len(phrases_start)))
        nst.append(len(phrases_start))
        flags.append(len(phrases))
#         for ii in ixs:
#             print(carticle[ii])
# #             print('***')
#             print(carticle[ii+1])
#             phrase = carticle[ii]
#             x = sum([1 for x in phrase if x.isdigit()])
#             phrase2 = carticle[ii+1]
#             x2 = sum([1 for x in phrase2 if x.isdigit()])
#             print(x, len(phrase), x2, len(phrase2))

#         chunks = aape.get_np_candidates(ixs, carticle, nlp, 1)
#         total_counts, total_counts_raw, table, tree_dict = aape.choose_popular_np_phrases(chunks)
#         df = pd.DataFrame(table, columns=['root', 'np', 'count'])
#         df['doi'] = doi
#         df_agg.append(df)

batch number 0


In [11]:
np_bag = None
np_dict = None

In [100]:
vcs, roots, np_bag, np_dict = aape.get_root0_stats(carticle, keywords, nlp, np_bag, 
                                                   np_dict, shift=0)

vcs1, roots1, np_bag, np_dict = aape.get_root0_stats(carticle, keywords, nlp, np_bag, 
                                                   np_dict, shift=1)

shifted_vcs = [vcs, vcs1]
tot_vcs = reduce(lambda x, y: x.merge(y[y.columns[:-1]], left_index=True, right_index=True, how='outer'), shifted_vcs)
tot_vcs.update(shifted_vcs[-1]['first_occ'])
tot_vcs = tot_vcs.fillna(0)
vcs.shape, vcs1.shape, tot_vcs.shape

((31, 4), (43, 4), (61, 7))

In [104]:
dff = aape.collect_full_stats(carticle, keywords, nlp)

len phrase gram dict 384
len vcs 1


In [106]:
thr_freq = 0.4
thr_len = 10
for doi in dois_batch[:-2]:
    carticle = articles_ds[doi]
    iis = aape.find_tablelike_phrases(carticle)
    for i in iis:
        print(carticle[i])
    stat.sort_values('freq', ascending=False)
#     chunks = aape.get_np_candidates(iis, crticle, nlp, 1)
#     total_counts, total_counts_raw, table, tree_dict = aape.choose_popular_np_phrases(chunks)

#     print(stat.sort_values('freq', ascending=False).head(5))

['Children', '.', '01', '.', '00', '.', '02', '.', '06', '6', '.']
['TABLE', '1', 'Differences', 'in', 'Committee', 'Membership', 'by', 'Directorial', 'Characteristicsa', 'Occupation', 'Type', 'Tenure', 'in', 'Years', 'Gender', 'Committee', 'Membership', 'Business', 'Nonbusiness', 'x2', '-', 'Outsider', '2', '0', '-', '6', '-', '10', '10', '2', 'Men', 'Women', 'x2', 'All', 'important', 'committees', 'Nonmember', '724', '222', '84', '.', '37', '543', '399', '259', '.', '28', '419', '217', '306', '137', '.', '96', '912', '35', '0', '.', '03', 'Member', '1', ',', '297', '884', '594', '1', ',', '575', '510', '664', '994', '2', ',', '103', '78', 'Audit', 'committee', 'Nonmember', '620', '475', '129', '.', '62', '1', '.', '076', '1', ',', '060', '578', '.', '13', 'b', '647', '557', '932', '15', '.', '81', '1', ',', '002', '58', '0', '.', '01', 'Member', '1', ',', '527', '473', '48', '894', '270', '310', '361', '846', '48', 'Nominating', 'committee', 'Nonmember', '808', '382', '29', '.', '86'

['Dealer', 'perceptions', 'of', 'influence', '1', '.', '000', '3', '.', '32', '.', '61', '2', '.']
['This', 'finding', 'does', 'not', 'support', 'theTABLE', '2', ':', 'Correlations', 'of', 'the', 'Variables', 'Used', 'in', 'the', 'Study', '1', '2', '3', '4', '5', '6', '7', '8', '9', '1', '.']
['Weighted', 'rewards', '-', '-', '51', '.', '42', '.', '59', '.', '72', '.', '07', '-', '31', '-', '15', '-', '28', '2', '.']
['U2', '0', '1', '-', 'U', 'U', 'U', '-W', '-', '1', 'x', '0', 'LO', '1', '04', 'g4', '04', '4', 'OC', 'C4', 'U', '0', '0', 'U', 'i114', '1', '-', '3', '0', '-', '0', 'U2', '0', '0', '0', '11', '-', 'U2', '.', '4', 'U', '4', '-', ',', '1', '14', '1', '4', 'tj', 'U', '0', '-', '1', '.', '.']
['U2', '1', 'U', '-', '.', 'r', '-', '4', '1', '4', ':', '3', 'En', '-', '4', 'to', '4', 'U2', '0', 'U', '-', '0', '1', '-', '7', '0', 'Lf', '0', 'W', '5', ':', '14', '-', '0', '0', '.', '-', 'A', 'tj', 'IT', '0', '04', 'O', 'U', '.', '4', '-', 'g', 'o', '0', '.', '4', '1', 'U', '.']
['

['Officer', 'in', 'trouble', '97', '396', '229', 'Breaking', 'and', 'entering', 'in', 'progress', '8', ',', '976', '18', ',', '212', '16', ',', '871', 'Cutting', 'or', 'stabbing', '313', '811', '495', 'Disturbance', '11', ',', '241', '29', ',', '193', '17', ',', '246', 'Domestic', 'trouble', '4', ',', '622', '10', ',', '592', '5', ',', '579', 'Drunk', '1', ',', '602', '3', ',', '050', '1', ',', '635', 'Homicide', '15', '39', '32', 'Man', 'with', 'gun', '369', '632', '353', 'Man', 'with', 'knife', '38', '74', '25', 'Prowlers', '87', '2', ',', '241', '4', ',', '066', 'Robbery', '1', ',', '567', '2', ',', '386', '943', 'Robbery', 'in', 'progress', '1', ',', '615', '1', ',', '585', '437', 'Shooting', '179', '568', '313', 'Shooting', ',', 'human', 'target', '529', '853', '487', 'Sex', 'crime', '600', '761', '490', 'Suicide', '47', '41', '36', 'Suicide', 'attempt', '103', '235', '172', 'Suspicious', 'car', '1', ',', '548', '3', ',', '417', '3', ',', '698', 'Vice', 'complaint', '77', '208', '

In [22]:
df0 = df0[['root', 'np', 'count', 'doi']].sort_values(['root', 'count'], 
                                                      ascending=[True, False]).reset_index(drop=True)
df0.to_csv('{0}np_data.csv.gz'.format(output_path), compression='gzip')

In [24]:
df0.shape, df0.drop_duplicates('np').shape

((431, 4), (411, 4))